In [0]:
import pyspark.sql.functions as F
data = [("Anderson","Sales","NY",90000,34,10000),
    ("Kenedy","Sales","CA",86000,56,20000),
    ("Billy","Sales","NY",81000,30,23000),
    ("Andy","Finance","CA",90000,24,23000),
    ("Mary","Finance","NY",99000,40,24000),
    ("Eduardo","Finance","NY",83000,36,19000),
    ("Mendes","Finance","CA",79000,53,15000),
    ("Keyth","Marketing","CA",80000,25,18000),
    ("Truman","Marketing","NY",91000,50,21000)
  ]

schema = ["emp_name","dep_name","state","salary","age","bonus"]
df = spark.createDataFrame(data=data, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- emp_name: string (nullable = true)
 |-- dep_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+--------+---------+-----+------+---+-----+
|emp_name|dep_name |state|salary|age|bonus|
+--------+---------+-----+------+---+-----+
|Anderson|Sales    |NY   |90000 |34 |10000|
|Kenedy  |Sales    |CA   |86000 |56 |20000|
|Billy   |Sales    |NY   |81000 |30 |23000|
|Andy    |Finance  |CA   |90000 |24 |23000|
|Mary    |Finance  |NY   |99000 |40 |24000|
|Eduardo |Finance  |NY   |83000 |36 |19000|
|Mendes  |Finance  |CA   |79000 |53 |15000|
|Keyth   |Marketing|CA   |80000 |25 |18000|
|Truman  |Marketing|NY   |91000 |50 |21000|
+--------+---------+-----+------+---+-----+



In [0]:
df.createTempView("tbl_teste")

In [0]:
%sql 
SELECT SUM(salary) as SALARIO, to_json(collect_list(map("DEPRTAMENTOS",dep_name,"PESSOAS",emp_name))) AS DEPARTAMENTOS  FROM tbl_teste GROUP BY state

SALARIO,DEPARTAMENTOS
444000,"[{""DEPRTAMENTOS"":""Sales"",""PESSOAS"":""Anderson""},{""DEPRTAMENTOS"":""Sales"",""PESSOAS"":""Billy""},{""DEPRTAMENTOS"":""Finance"",""PESSOAS"":""Mary""},{""DEPRTAMENTOS"":""Finance"",""PESSOAS"":""Eduardo""},{""DEPRTAMENTOS"":""Marketing"",""PESSOAS"":""Truman""}]"
335000,"[{""DEPRTAMENTOS"":""Sales"",""PESSOAS"":""Kenedy""},{""DEPRTAMENTOS"":""Finance"",""PESSOAS"":""Andy""},{""DEPRTAMENTOS"":""Finance"",""PESSOAS"":""Mendes""},{""DEPRTAMENTOS"":""Marketing"",""PESSOAS"":""Keyth""}]"


In [0]:
df.groupBy(F.col("dep_name")).sum("salary").show(truncate=False)

+---------+-----------+
|dep_name |sum(salary)|
+---------+-----------+
|Sales    |257000     |
|Finance  |351000     |
|Marketing|171000     |
+---------+-----------+



In [0]:
df.groupBy(F.col("dep_name")).count().show(truncate=False)

+---------+-----+
|dep_name |count|
+---------+-----+
|Sales    |3    |
|Finance  |4    |
|Marketing|2    |
+---------+-----+



In [0]:
df.groupBy(F.col("dep_name")).min("salary").show(truncate=False)

+---------+-----------+
|dep_name |min(salary)|
+---------+-----------+
|Sales    |81000      |
|Finance  |79000      |
|Marketing|80000      |
+---------+-----------+



In [0]:
df.groupBy(F.col("dep_name")).max("salary").show(truncate=False)

+---------+-----------+
|dep_name |max(salary)|
+---------+-----------+
|Sales    |90000      |
|Finance  |99000      |
|Marketing|91000      |
+---------+-----------+



In [0]:
df.groupBy(F.col("dep_name")).mean("salary").show(truncate=False)

+---------+-----------------+
|dep_name |avg(salary)      |
+---------+-----------------+
|Sales    |85666.66666666667|
|Finance  |87750.0          |
|Marketing|85500.0          |
+---------+-----------------+



In [0]:
df.groupBy(F.col("dep_name")).avg("salary").show(truncate=False)

+---------+-----------------+
|dep_name |avg(salary)      |
+---------+-----------------+
|Sales    |85666.66666666667|
|Finance  |87750.0          |
|Marketing|85500.0          |
+---------+-----------------+



In [0]:
df.groupBy(F.col("dep_name"),F.col("state")).sum("salary","bonus").show(truncate=False)

+---------+-----+-----------+----------+
|dep_name |state|sum(salary)|sum(bonus)|
+---------+-----+-----------+----------+
|Sales    |NY   |171000     |33000     |
|Sales    |CA   |86000      |20000     |
|Finance  |CA   |169000     |38000     |
|Finance  |NY   |182000     |43000     |
|Marketing|NY   |91000      |21000     |
|Marketing|CA   |80000      |18000     |
+---------+-----+-----------+----------+



In [0]:
df.groupBy(F.col("dep_name")) \
.agg(F.sum("salary").alias("Sum_salary"), \
    F.avg("salary").alias("Avg_salary"), \
    F.sum("bonus").alias("Sum_bonus"), \
    F.max("bonus").alias("Max_bonus"), \
    F.min("bonus").alias("Min_bonus")    
    ).show(truncate=False)

+---------+----------+-----------------+---------+---------+---------+
|dep_name |Sum_salary|Avg_salary       |Sum_bonus|Max_bonus|Min_bonus|
+---------+----------+-----------------+---------+---------+---------+
|Sales    |257000    |85666.66666666667|53000    |23000    |10000    |
|Finance  |351000    |87750.0          |81000    |24000    |15000    |
|Marketing|171000    |85500.0          |39000    |21000    |18000    |
+---------+----------+-----------------+---------+---------+---------+



In [0]:
df.groupBy(F.col("dep_name")) \
.agg(F.sum("salary").alias("Sum_salary"), \
    F.avg("salary").alias("Avg_salary"), \
    F.sum("bonus").alias("Sum_bonus"), \
    F.max("bonus").alias("Max_bonus"), \
    F.min("bonus").alias("Min_bonus")    
    ).where(F.col("Sum_salary") >= 79000 ).show(truncate=False)

+---------+----------+-----------------+---------+---------+---------+
|dep_name |Sum_salary|Avg_salary       |Sum_bonus|Max_bonus|Min_bonus|
+---------+----------+-----------------+---------+---------+---------+
|Sales    |257000    |85666.66666666667|53000    |23000    |10000    |
|Finance  |351000    |87750.0          |81000    |24000    |15000    |
|Marketing|171000    |85500.0          |39000    |21000    |18000    |
+---------+----------+-----------------+---------+---------+---------+

